In [1]:
import matplotlib.pyplot as plt
import numpy as np
from Environment.E_commerce import *
from Algorithms.Conv_rates_Learner import *
from Algorithms.Greedy_Learner import *
from Algorithms.TS_Learner import *
from Algorithms.TS_Learner_poisson import *
from Algorithms.UCB_Learner import *
from Algorithms.estimate_probabilities import estimate_probabilities

In [7]:
P1 = Product(0,[9,12,13,14.5],[ 1.,  4,  5., 6.5])
P2 = Product(1,[20,22.5,23,24.5],[ 4.,  6.5, 7., 8.5])
P3 = Product(2,[30,31.5,34,34.5],[ 6.,  7.5, 10., 10.5])
P4 = Product(3,[40,42.5,43,46.5],[ 8., 10.5, 11., 14.5])
P5 = Product(4,[50,51.5,53,54.5],[10., 11.5, 13., 14.5])

products = [P1,P2,P3,P4,P5]
margins_matrix = np.zeros((5,4))
expected_units_sold_per_product = np.array([2,1,3,3,1]) #these are the lambdas defined in the visit function
for i in range(5):
    for j in range(4):
        margins_matrix[i,j] = products[i].margins_list[j]
E = E_commerce(binary_features=0)
E.set_lambda(0.5)
E.set_products(products)

#Ecommerce graph weights are uncertain, for each user they are different, but the slots are fixed by the Ecommerce
E.graph = np.array([[0. , 0.5, 0. , 1. , 0. ],[1. , 0. , 0. , 0.5, 0. ],[0. , 1. , 0. , 0.5, 0. ],[0. , 0.5, 1. , 0. , 0. ],[1. , 0. , 0. , 0.5, 0. ]])

In [8]:
n_arms = 4
num_users = 10
n_days = 100
fixed_alpha = 1
fixed_weights = 1
fixed_units = 1
num_experiments = 1
#prob_matrix = np.random.uniform(0.0, 0.1, (n_nodes, n_nodes))

ts_rewards_per_experiment = []
ts_pulls_per_arm_per_experiment = []

for e in range(num_experiments):

    env = Environment(n_arms, E, margins_matrix, num_users, fixed_alpha,fixed_weights,fixed_units)
    ts_learner = TS_poisson(n_arms=n_arms)

    for d in range(n_days):
        pulled_arm = ts_learner.pull_arm(env.margins_matrix)
        reward = env.round(pulled_arm)
        ts_learner.update(pulled_arm, reward, env.clicks_current_day, env.purchases_current_day, env.daily_units)

    ts_rewards_per_experiment.append(ts_learner.collected_rewards)
    ts_pulls_per_arm_per_experiment.append(ts_learner.counter_per_arm)



In [9]:
node_index = 0
n_nodes = 5
dataset = env.E.time_history
estimated_probab = estimate_probabilities(dataset = env.E.time_history, node_index = node_index, n_nodes=5)
estimated_probab = (estimated_probab).reshape((5, 1))
for node_i in range(1,5):
    new_vector = estimate_probabilities(dataset = env.E.time_history, node_index = node_i, n_nodes=5)
    new_vector = new_vector.reshape((5,1))
    estimated_probab = np.append(estimated_probab,new_vector, axis=1)

/var/folders/gq/dy2890t935s_c5vn0fg388480000gn/T/ipykernel_5473/3149403130.py:27: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if len(idx_v_active) > 0 and (idx_v_active < idx_w_active or len(idx_w_active) == 0):
/var/folders/gq/dy2890t935s_c5vn0fg388480000gn/T/ipykernel_5473/3149403130.py:29: RuntimeWarning: invalid value encountered in true_divide
  estimated_prob = credits / occurr_v_active


In [10]:
estimated_probab

array([[0.        , 0.        , 0.        , 0.07792208, 0.        ],
       [0.43046358, 0.        , 0.        , 0.16935484, 0.        ],
       [0.01239669, 0.25663717, 0.        , 0.07      , 0.        ],
       [0.00352113, 0.17424242, 0.17333333, 0.        , 0.        ],
       [0.05      , 0.        , 0.        , 0.21      , 0.        ]])

In [11]:
u0 = HomogeneousUsers(1, 1)
u0.P * env.E.graph

array([[0.        , 0.01495478, 0.        , 0.27068635, 0.        ],
       [0.4918155 , 0.        , 0.        , 0.20504501, 0.        ],
       [0.        , 0.32939066, 0.        , 0.09244458, 0.        ],
       [0.        , 0.23745063, 0.33253022, 0.        , 0.        ],
       [0.04359364, 0.        , 0.        , 0.20908477, 0.        ]])